<a href="https://colab.research.google.com/github/danielriosgarza/AiSchool/blob/main/content/DanielGarza/notebooks/kinetic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!git clone https://github.com/danielriosgarza/AiSchool.git

%cd AiSchool/content/DanielGarza/
!pip install .[all]

# **Kinetic model environment**

The `kinetic model` environment allows us to simulate microbial communities and their environments using ordinary differential equations.

To illustrate how it works, we build a simplified examplt with *E. coli* subpopulations.

In this example, there are three subpopulations: two active and one inactive.

- **Subpopulation A** consumes glucose and produces acetate (overflow).

- **Subpopulation B** consumes acetate and produces CO2. Both subpopulations can transition to the inactive state.


<div style="height:24px;"></div>

```css
Glucose ---> [Subpopulation A] ---> Acetate
Acetate ---> [Subpopulation B] ---> CO2

[Subpopulation A] ---> [Inactive]
[Subpopulation B] ---> [Inactive]
```

<div style="height:24px;"></div>


#### **Metabolites & Metabolome**

First, we create three `Metabolite` objects—glucose, acetate, and CO2—and group them in a `Metabolome`.


In [3]:
#import packages for the model and visualization
from kinetic_model import Metabolite, Metabolome
from kinetic_model.visualize import GraphSpecBuilder, CytoscapeExporter
import json
#to make sure plotly renders in the notebook
import plotly.io as pio
pio.renderers.default = "colab"
# Display Cytoscape visualization
from ipycytoscape import CytoscapeWidget
import ipywidgets as widgets



#create metabolites
glucose = Metabolite(name = "glucose", concentration = 5.0, formula ={'C': 6, 'H': 12, 'O': 6}, color = '#ff0000')
acetate = Metabolite(name = "acetate", concentration = 0.0, formula ={'C': 2, 'H': 4, 'O': 3}, color = '#003eff')
co2 = Metabolite(name = "co2", concentration = 0.0, formula ={'C': 1, 'O': 2}, color = '#00B8FF')

In [4]:
#create metabolome
metabolome = Metabolome(metabolites = [glucose, acetate, co2])

#visualize it
metabolome.make_plot()



We can also visualize the model components in Cytoscape

In [5]:

#cycle through a kson file
model = json.loads(metabolome.to_json(full_model=True))

# Build the graph specification
builder = GraphSpecBuilder()
graph_spec = builder.build_from_json(model)

# Export to Cytoscape
exporter = CytoscapeExporter()
cytoscape_data = exporter.export(graph_spec, layout="nice", show_edge_labels=True)

# Create the viewer
viewer = CytoscapeWidget()
viewer.graph.add_graph_from_json(cytoscape_data['elements'])
viewer.set_style(cytoscape_data['style'])
viewer.set_layout(name="preset")

# Display
display(viewer)

CytoscapeWidget(cytoscape_layout={'name': 'preset'}, cytoscape_style=[{'selector': 'node', 'style': {'label': …

#### **Fedding terms**

Support for third party widgets will remain active for the duration of the session. To disable support:

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

`Feeding term` objects encode how microbial subpopulations consume metabolites and release byproducts. They are defined independently and later linked to subpopulations.


- If the `feeding term` requires more than one input metabolite, the relation is an `AND`:


```css
[metabolite A] + [metabolite C] ---> [metabolite B]
```


- If a subpopulations has more than one feeding term, the relation is an `OR`:

```css
[metabolite A] ---> [metabolite B] + [metabolite C]

[metabolite D] ---> metabolite E]
```

- A `Boost` is an  `AND` combined with an `OR`.

- Sequential (`XOR`) requires alternative subpopulations.


In the *E. coli* example, there are two feeding terms.

In [7]:
#Basic usage with simple metabolome:
from kinetic_model import FeedingTerm
# Create feeding term: consumes glucose, produces lactate
feeding_term_1 = FeedingTerm(
    id="glucose_to_acetate",
    metDict={"glucose": [1.0, 1.0], "acetate": [-1.0, 0.0]},
    metabolome=metabolome)

feeding_term_2 = FeedingTerm(
    id="acetate_to_co2",
    metDict={"acetate": [1.0, 1.0], "co2": [-1.0, 0.0]},
    metabolome=metabolome
)


We can perform basic calculations of metabolic rates using the current metabolite concentrations defined in the `metabolome` object.

In [8]:
# Get current concentrations
concentrations = metabolome.get_concentration()
# Calculate growth and metabolism rates
growth_contribution = feeding_term_1.intrinsicGrowth(concentrations)
metabolism_rates = feeding_term_1.intrinsicMetabolism(concentrations)

print(f"growth_contribution ft 1: {growth_contribution:.3f}")
print(f"Metabolism rates ft 1: {metabolism_rates}")


# Calculate growth and metabolism rates
growth_contribution = feeding_term_2.intrinsicGrowth(concentrations)
metabolism_rates = feeding_term_2.intrinsicMetabolism(concentrations)

print(f"growth_contribution ft 2: {growth_contribution:.3f}")
print(f"Metabolism rates ft 2: {metabolism_rates}")

growth_contribution ft 1: 0.833
Metabolism rates ft 1: [ 0.83333333 -0.         -0.83333333]
growth_contribution ft 2: 0.000
Metabolism rates ft 2: [-0.  0. -0.]


Let’s visualize the glucose feeding term.

In [9]:
model = json.loads(feeding_term_1.to_json(full_model=True))

# Build the graph specification
builder = GraphSpecBuilder()
graph_spec = builder.build_from_json(model)

# Export to Cytoscape
exporter = CytoscapeExporter()
cytoscape_data = exporter.export(graph_spec, layout="nice", show_edge_labels=True)

# Create the viewer
viewer = CytoscapeWidget()
viewer.graph.add_graph_from_json(cytoscape_data['elements'])
viewer.set_style(cytoscape_data['style'])
viewer.set_layout(name="preset")

# Display
display(viewer)

CytoscapeWidget(cytoscape_layout={'name': 'preset'}, cytoscape_style=[{'selector': 'node', 'style': {'label': …

#### **Subpopulations**

Subpopulations represent microbial phenotypes—defined by their feeding terms—and their state (e.g., active or inactive)

In [10]:
from kinetic_model import Subpopulation


# Create subpopulation
subpop_1 = Subpopulation(
    name="glucose_ecoli",
    count=1.0,
    species="E. coli",
    mumax=0.5,
    feedingTerms=[feeding_term_1],
    pHopt=7.0,
    pH_sensitivity_left=1.0,
    pH_sensitivity_right=1.0,
    Topt=37.0,
    tempSensitivity_left=1.0,
    tempSensitivity_right=1.0
    )


subpop_2 = Subpopulation(
    name="acetate_ecoli",
    count=1.0,
    species="E. coli",
    mumax=0.3,
    feedingTerms=[feeding_term_2],
    pHopt=7.0,
    pH_sensitivity_left=1.0,
    pH_sensitivity_right=1.0,
    Topt=37.0,
    tempSensitivity_left=1.0,
    tempSensitivity_right=1.0
    )

subpop_3 = Subpopulation(
    name="inactive_ecoli",
    count=1.0,
    species="E. coli",
    mumax=0.0,
    feedingTerms=[],
    state="inactive",
    pHopt=7.0,
    pH_sensitivity_left=1.0,
    pH_sensitivity_right=1.0,
    Topt=37.0,
    tempSensitivity_left=1.0,
    tempSensitivity_right=1.0
    )




# Access properties
print(f"Growth rate: {subpop_1.mumax}")
print(f"pH sensitivity at pH 7.0: {subpop_1.pHSensitivity(7.0):.3f}")
print(f"Temperature sensitivity at 37°C: {subpop_1.tempSensitivity(37.0):.3f}")

# Calculate growth and metabolism with concentration vector
growth_rate = subpop_1.intrinsicGrowth(concentrations)
metabolism_rates = subpop_1.intrinsicMetabolism(concentrations)



Growth rate: 0.5
pH sensitivity at pH 7.0: 1.000
Temperature sensitivity at 37°C: 1.000


#### **Bacteria and Microbiomes**

A species is represented by a `Bacteria` object, defined as a collection of subpopulations and the transition terms between them. Transitions depend on the environment and are specified as boolean or threshold functions, each combined with a rate.

For example, we can define that the glucose-consuming E. coli subpopulation transitions to the acetate-consuming subpopulation when glucose falls below $1 mM$, at a rate of $0.001.$ Conversely, when glucose rises above $1 mM$, the acetate-consuming subpopulation transitions back to glucose consumption at a rate of $0.01$.

Microbiomes are encoded as collections of species.

In [11]:
from kinetic_model import Bacteria, Microbiome, evaluate_transition_condition

# Create bacteria with conditional transitions based on resource availability

cond1 = evaluate_transition_condition('glucose <1 and acetate > 1', metabolome)
cond2  = evaluate_transition_condition('glucose > 1 and acetate < 1', metabolome)

ecoli = Bacteria(
    species="E. coli",
    subpopulations={
        "glucose_consumer": subpop_1,
        "acetate_consumer": subpop_2,
        "inactive": subpop_3
        },
    connections={
        "glucose_consumer": [["acetate_consumer", cond1, 0.001], ["inactive", "", 0.00001]],
        "acetate_consumer": [["glucose_consumer", cond2, 0.01], ["inactive", "", 0.0000001]],
        }
    )

# Access properties
print(f"Species: {ecoli.species}")
print(f"Subpopulations: {list(ecoli.subpopulations.keys())}")
print(f"Connections: {ecoli.connections}")


# ===== CREATE MICROBIOME =====
microbiome = Microbiome("E_coli_monoculture", {"E_coli": ecoli})

Species: E. coli
Subpopulations: ['glucose_consumer', 'acetate_consumer', 'inactive']
Connections: {'glucose_consumer': [['acetate_consumer', <function evaluate_transition_condition.<locals>.transition_function at 0x7a6842752de0>, 0.001], ['inactive', '', 1e-05]], 'acetate_consumer': [['glucose_consumer', <function evaluate_transition_condition.<locals>.transition_function at 0x7a6842752ac0>, 0.01], ['inactive', '', 1e-07]]}


Let’s visualize it.

In [12]:
# Get complete model structure directly - no manual structure creation needed!
model = json.loads(microbiome.to_json(full_model=True))

# Build the graph specification
builder = GraphSpecBuilder()
graph_spec = builder.build_from_json(model)

# Export to Cytoscape
exporter = CytoscapeExporter()
cytoscape_data = exporter.export(graph_spec, layout="nice", show_edge_labels=True)

# Create the viewer
viewer = CytoscapeWidget()
viewer.graph.add_graph_from_json(cytoscape_data['elements'])
viewer.set_style(cytoscape_data['style'])
viewer.set_layout(name="preset")

# Display
display(viewer)

CytoscapeWidget(cytoscape_layout={'name': 'preset'}, cytoscape_style=[{'selector': 'node', 'style': {'label': …

#### **Environment**

The `Environment` object encodes the system’s environmental conditions—currently pH, temperature, and stirring.

- `pH` influences microbial growth through each subpopulation’s pH sensitivity. It can be set as a fixed value or defined as a function of metabolite concentrations.

- `Temperature` is specified as a fixed value and affects growth according to the temperature sensitivity parameter.

- `Stirring controls` how evenly cells and metabolites are distributed. In a well-mixed system (stirring = 1), concentrations match their true values exactly. When stirring is zero, observed concentrations are taken as random samples of the true values.

In [13]:
from kinetic_model import Environment, pH, Stirring, Temperature
# Create components
ph_obj = pH(metabolome, intercept=7.0, met_dictionary={}) #pH will be fixed at 7
stirring_obj = Stirring(rate=0.8, base_std=0.05)
temp_obj = Temperature(37.0)

# Create environment
environment = Environment(ph_obj, stirring_obj, temp_obj)
# Apply all environmental factors at once
concentrations = metabolome.get_concentration()  # glucose concentration
result = environment.apply_all_factors(concentrations)
# Or access individual components
current_ph = environment.pH.compute_pH(concentrations)
stirred_value = environment.stirring.apply_stirring(10.0)
current_temp = environment.temperature.temperature


print(f"pH: {current_ph}")
print(f"Stirred value: {stirred_value}")
print(f"Temperature: {current_temp}")

pH: 7.0
Stirred value: 10.011256780484395
Temperature: 37.0


#### **Control Pulse**

The `Pulse` object defines the environmental conditions during a fixed time interval. It specifies what enters or leaves the reactor (metabolites or cells), either as a single event at the start of the interval or continuously throughout it. The `Pulse` also sets the environmental parameters (`pH`, `temperature`, `stirring`) for that period.

In [14]:
from kinetic_model import Pulse
pulse = Pulse(
        t_start=0.0,
        t_end=120.0,   # 120 hours
        n_steps=1000,
        qin=0.0,
        qout=0.0,
        vin=0.0,
        vout=0.0,
        environment=environment
    )

#### **Reactor**

The `Reactor` object is where microbial growth is simulated. It contains a `metabolome`, a `microbiome`, and a sequence of control pulses that define `pH`, `temperature`, `stirring`, and both continuous and non-continuous `feeding`.

In [15]:
from kinetic_model import Reactor
reactor = Reactor(
        microbiome=microbiome,
        metabolome=metabolome,
        pulses=[pulse],
        volume=15.0,
    )

#### **Simulation**

We can now simulate the reactor and plot the results.

In [16]:
reactor.set_fast_simulation_mode()
reactor.simulate()
reactor.make_plot()

Setting fast simulation mode...
Reduced pulse 0.0-120.0: 1000 → 250 steps
Total steps: 1000 → 250
Estimated speedup: 4.0x
Fast mode enabled: RK45 solver, relaxed tolerances (1e-2)


#### **Example gut synthetic community**

We can also load the entire model at once from a JSON file.

In [17]:
from kinetic_model import ModelFromJson


json_file = "/content/AiSchool/content/DanielGarza/modelTemplates/bh_bt_ri_complete_model_export.json"
model = ModelFromJson(json_file)
model_feed = ModelFromJson(json_file)
pulse = Pulse(
    t_start=0.0,
    t_end=1200.0,
    n_steps=1000,
    qin=0.039*15, #continous feed
    qout=0.039*15,#continous feed
    vin=0.0,#non-continuos feed
    vout=0.0,#non-continuos feed
    environment=model.environment,
    continuous_feed_metabolome = model_feed.metabolome,
    )

reactor = Reactor(
    microbiome=model.microbiome,
    metabolome=model.metabolome,
    pulses=[pulse],
    volume=15.0,
    )



In [18]:
# Load the model
with open(json_file, 'r') as f:
    mod = json.load(f)
# Build the graph specification
builder = GraphSpecBuilder()
graph_spec = builder.build_from_json(mod)

# Export to Cytoscape
exporter = CytoscapeExporter()
cytoscape_data = exporter.export(graph_spec, layout="nice", show_edge_labels=True)

# Create the viewer
viewer = CytoscapeWidget()
viewer.graph.add_graph_from_json(cytoscape_data['elements'])
viewer.set_style(cytoscape_data['style'])  # Use 'style' instead of 'stylesheet'
viewer.set_layout(name="preset")

# Display
display(viewer)

CytoscapeWidget(cytoscape_layout={'name': 'preset'}, cytoscape_style=[{'selector': 'node', 'style': {'label': …

In [19]:
reactor.set_fast_simulation_mode()
reactor.simulate()
reactor.make_plot()

Setting fast simulation mode...
Reduced pulse 0.0-1200.0: 1000 → 250 steps
Total steps: 1000 → 250
Estimated speedup: 4.0x
Fast mode enabled: RK45 solver, relaxed tolerances (1e-2)


Support for third party widgets will remain active for the duration of the session. To disable support:

In [20]:
from google.colab import output
output.disable_custom_widget_manager()